In [1]:
# Libraries
import open3d as o3d
import viser
import torch
import numpy as np
import cv2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#@title Importing the Clusters with It's radiances
slices = torch.load("/workspace/FruitProposal/attachment/RadianceCloud/slice_points.pt")
print(slices.keys())

planes  = slices['plane']
points  = slices['points']
rgbs    = slices['rgb']
centers = slices['slice_center']
corners = slices['slice_corner']


dict_keys(['points', 'plane', 'rgb', 'slice_corner', 'slice_center', 'max_points'])


In [3]:
#server = viser.ViserServer()

base_img = np.zeros(())

_points = []
_colors = []

for k, point, plane, rgb in zip(range(len(points)),points, planes, rgbs):
    # Create a point cloud
    _slice = o3d.geometry.PointCloud()
    _slice.points = o3d.utility.Vector3dVector(plane)
    _slice.colors = o3d.utility.Vector3dVector(rgb)
    
    # Normalize the point cloud xn = x/z, yn = y/z, z = 1
    _slice.points = o3d.utility.Vector3dVector(np.asarray(_slice.points) / np.asarray(_slice.points)[:, 2:3])

    _points.append(np.asarray(_slice.points))
    _colors.append(np.asarray(_slice.colors))


In [4]:
def slice_resolution(num_points, min_size=(0,0)):
    Wp = int(round((num_points)**0.5))
    Hp = int(round(num_points / Wp))
    Wp = max(Wp, min_size[0])
    Hp = max(Hp, min_size[1])
    return Wp, Hp

In [5]:
f = lambda x : x.shape[0]

max_points = max(map(f, _points))
print("Max points per slice:", max_points)

# Obtaining image resolution
W = np.sqrt(max_points*2).astype(int)

W = 512

print("Image resolution x", W, "y", W)

print("Points capacity", W*W)


Max points per slice: 607
Image resolution x 512 y 512
Points capacity 262144


In [6]:
def coordToPixel(coordW, extrinsic, intrinsic) -> np.ndarray:
    """
    Convert world coordinates to pixel coordinates.
    """
    coordW = np.array(coordW)
    coordW = np.concatenate((coordW, np.ones((coordW.shape[0], 1))), axis=1)
    coordC = (extrinsic @ coordW.T).T[:, :3]
    coordP = (intrinsic @ coordC.T).T
    coordP = coordP[:, :2] / coordP[:, 2:3]
    return coordP.astype(int)

In [ ]:
# Precompute intrinsics once
z_cam = np.array([0, 0, -1])
W = H = 512
cx = W / 2
cy = H / 2
fxy = W
intrinsic = np.array([
    [fxy,   0,  cx],
    [  0, fxy,  cy],
    [  0,   0,   1]
])

for k, (pts, plane, rgb, corners_k, center_k) in enumerate(
        zip(points, planes, rgbs, corners, centers, strict=False)):

    # 1) elevar la slice por encima de Z=0
    z0      = plane[:, 2].min()
    shift_z = max(0.05 - z0, 0.0)

    # 2) calcular R usando esquinas desplazadas
    _corners = np.array(corners_k)
    _corners[:, 2] += shift_z
    e       = _corners[0] - _corners[1]
    e_norm  = np.linalg.norm(e)
    y_cam   = e / e_norm
    x_cam   = np.cross(y_cam, z_cam);  x_cam /= np.linalg.norm(x_cam)
    y_cam   = np.cross(z_cam, x_cam);  y_cam /= np.linalg.norm(y_cam)
    R       = np.column_stack((x_cam, y_cam, z_cam))

    # 3) posición inicial de la cámara
    center_shift = center_k.copy()
    center_shift[2] += shift_z
    Ccam = center_shift + np.array([0, 0, e_norm])

    # --- ajuste fino: centrar la proyección del centro del slice ---
    Xc_center = R @ (center_shift - Ccam)
    uvw_c     = intrinsic @ Xc_center
    u_c, v_c  = uvw_c[0]/uvw_c[2], uvw_c[1]/uvw_c[2]
    delta_u, delta_v = cx - u_c, cy - v_c

    # pasar de píxeles a desplazamiento en el espacio de cámara
    # un píxel en x corresponde a 1/fxy unidades en x_cam, igual para y
    Ccam = Ccam - (delta_u / fxy) * x_cam - (delta_v / fxy) * y_cam

    # 4) preparar lienzo
    void_image = np.zeros((H, W, 3), dtype=np.uint8)

    # 5) proyectar cada punto
    for pt, col in zip(plane, rgb):
        p = pt.copy();  p[2] += shift_z
        Xw_cam0 = p - Ccam
        Xc      = R @ Xw_cam0
        if Xc[2] <= 0:
            continue

        uvw = intrinsic @ Xc
        u, v = uvw[0]/uvw[2], uvw[1]/uvw[2]
        ui, vi = int(round(u)), int(round(v))
        if 0 <= ui < W and 0 <= vi < H:
            void_image[vi, ui] = (np.array(col)*255).astype(np.uint8)[::-1]

    # 6) dilatar y guardar
    kernel = np.ones((3,3), np.uint8)
    out = cv2.dilate(void_image, kernel, iterations=5)
    cv2.imwrite(f"/workspace/FruitProposal/attachment/RadianceCloud/images/slice_{k}.png", out)


Slice 0 center → (u,v)=(256.00,256.00) — expected (256.00,256.00)
Slice 1 center → (u,v)=(256.00,256.00) — expected (256.00,256.00)
Slice 2 center → (u,v)=(256.00,256.00) — expected (256.00,256.00)
Slice 3 center → (u,v)=(256.00,256.00) — expected (256.00,256.00)
Slice 4 center → (u,v)=(256.00,256.00) — expected (256.00,256.00)
Slice 5 center → (u,v)=(256.00,256.00) — expected (256.00,256.00)
Slice 6 center → (u,v)=(256.00,256.00) — expected (256.00,256.00)
Slice 7 center → (u,v)=(256.00,256.00) — expected (256.00,256.00)
Slice 8 center → (u,v)=(256.00,256.00) — expected (256.00,256.00)
